In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [ ]:
%pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Usin

In [ ]:
%pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    load_in_4bit=True,
    #attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device = 'cuda:0'

In [ ]:
df = pd.read_csv('stockmarket.csv')

In [ ]:
df.head()

,Company,title,description,Date,Up/Down
0,Apple,Apple’s time travel comedy Time Bandits starts...,Apple TV Plus has revealed the release date fo...,2024-05-21,Up
1,Apple,Hands-on with the Surface Laptop on Arm,The 7th edition Surface Laptop comes with Qual...,2024-05-21,Up
2,Apple,The 10th-gen iPad drops to its lowest price ever,Some fancy new iPads have just hit store shelv...,2024-05-21,Up
3,Apple,Sonos Finally Has Headphones and We’re Excited,"It’s been a long time coming, but that pair of...",2024-05-21,Up
4,Apple,Microsoft’s All-New ‘Copilot+’ PCs and Surface...,"Microsoft smells blood in the water, and it se...",2024-05-21,Up


In [ ]:
df.groupby(['Company','Date']).count()

title  description  Up/Down
Company Date                                   
Amazon  2024-05-21     96           96       96
        2024-05-22     97           97       97
        2024-05-23     96           96       96
        2024-05-24     93           93       93
        2024-05-28     95           95       95
...                   ...          ...      ...
Tesla   2024-06-12     90           90       90
        2024-06-13     88           88       88
        2024-06-14     91           91       91
        2024-06-17     91           91       91
        2024-06-18     97           97       97

[140 rows x 3 columns]

In [ ]:
results = []
for comp in df['Company'].unique():
    df_comp = df[df['Company'] == comp]
    for date in df['Date'].unique():
        f5 = df_comp[df_comp['Date'] == date].iloc[0:15]
        text_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful AI assistant for stock market tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

        I am going to provide you with some news blurbs about {comp} from today, and a multiple choice question at the end. Please memorize the news, then answer the question. Here are the blurbs:"""
        for i in range(15):
            text_prompt += f"1) {f5['title'].iloc[i]} : {f5['description'].iloc[i]}\n"
        text_prompt += f"\nNow here is the question: Based on your understanding of the news, determine whether {comp}'s stock will go up or down by open tomorrow. Please respond with either (A) - UP or (B) - DOWN.<|eot_id|><|start_header_id|>assistant<|end_header_id|>The correct answer is \n"
        input_ids = tokenizer(text_prompt, return_tensors='pt').input_ids
        input_ids = input_ids.to(device)
        out = model.generate(input_ids, max_new_tokens=2, do_sample=False, temperature=0)
        pred = tokenizer.decode(out[0])[-10:]
        print(comp, date, pred)
        results.append([comp, date, pred])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_m

Apple 2024-05-21 er is 
(B)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Apple 2024-05-22 er is 
(B)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Nvidia 2024-06-13 er is 
(A)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nvidia 2024-06-14 er is 
(B)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nvidia 2024-06-17 er is 
(B)
Nvidia 2024-06-18 wer is 
A)


In [ ]:
results = [
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'A',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
    'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B',
    'B', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B', 'A'
]

In [ ]:
labels = []
for comp in df['Company'].unique():
    df_comp = df[df['Company'] == comp]
    for date in df_comp['Date'].unique():

        label = df_comp[df_comp['Date'] == date]['Up/Down'].iloc[0]
        labels.append(label)
print(labels)


['Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Down', 'Up', 'Up', 'Up', 'Down', 'Up', 'Down', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Down', 'Down', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Up', 'Up', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Down', 'Down', 'Down', 'Down', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Up', 'Down', 'Up', 'Up', 'Up', 'Up', 'Down', 'Down', 'Up', 'Up', 'Up', 'Down', 'Down', 'Up', 'Down', 'Up', 'Up', 'Up', 'Down', 'Up']


In [ ]:
labels = [1 if x == 'Up' else 0 for x in labels]

In [ ]:
labels

[1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1]

In [ ]:
results

[['Apple', '2024-05-21', 'er is \n(B)'],
 ['Apple', '2024-05-22', 'er is \n(B)'],
 ['Apple', '2024-05-23', 'er is \n(B)'],
 ['Apple', '2024-05-24', 'er is \n(B)'],
 ['Apple', '2024-05-28', 'er is \n(B)'],
 ['Apple', '2024-05-29', 'er is \n(B)'],
 ['Apple', '2024-05-30', 'er is \n(B)'],
 ['Apple', '2024-05-31', 'er is \n(B)'],
 ['Apple', '2024-06-03', 'er is \n(B)'],
 ['Apple', '2024-06-04', 'er is \n(B)'],
 ['Apple', '2024-06-05', 'er is \n(B)'],
 ['Apple', '2024-06-06', 'er is \n(B)'],
 ['Apple', '2024-06-07', 'er is \n(B)'],
 ['Apple', '2024-06-10', 'er is \n(A)'],
 ['Apple', '2024-06-11', 'er is \n(B)'],
 ['Apple', '2024-06-12', 'er is \n(B)'],
 ['Apple', '2024-06-13', 'er is \n(A)'],
 ['Apple', '2024-06-14', 'er is \n(B)'],
 ['Apple', '2024-06-17', 'er is \n(B)'],
 ['Apple', '2024-06-18', 'er is \n(B)'],
 ['Amazon', '2024-05-21', 'er is \n(B)'],
 ['Amazon', '2024-05-22', 'er is \n(B)'],
 ['Amazon', '2024-05-23', 'er is \n(B)'],
 ['Amazon', '2024-05-24', 'er is \n(B)'],
 ['Amazon', 

In [ ]:
res = [1 if 'A' in x[2] else 0 for x in results]

In [ ]:
res

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1]

In [ ]:
len(y_true)

140

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Replace these with your actual predictions and true labels
y_true = labels  # True labels
y_pred = res  # Predicted labels

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Accuracy: 0.41
Precision: 0.43
Recall: 0.17
F1 Score: 0.24


In [ ]:
torch.cuda.empty_cache()